<a href="https://colab.research.google.com/github/simongiando/ImageProcessing/blob/main/basic/IP_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import geopandas as gpd
import ee
from IPython.display import Image
import folium

In [3]:
!pip install --upgrade geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 16.7 MB 50.7 MB/s 
     |████████████████████████████████| 6.3 MB 41.4 MB/s 


In [7]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-Kkh8IxsWM7OJtRZVcvHiUL7Za-_OzJ-E6aokQESeQs&tc=AEyqPs118yxRVOD5D5EfGSdIlVN2LS6Q4wFBZ6c3R_0&cc=PKN9Tzqu1ZKGRyYnFilLYOcSCEkDYSS2IwE3QdDQk4c

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJpDBOAJuB9bDn2X1bM5Hka7PZwSIQOfW5vovwmfn3fmtq4OFvXntno

Successfully saved authorization token.


In [15]:
lat = 34.5195 
lon = -18.8543

poi = ee.Geometry.Point (lat,lon)

start = '2018-10-01'
end = '2019-01-31'

In [16]:
landsat = ee.ImageCollection ("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds (poi)\
            .filterDate(start,end)

In [17]:
print (landsat.size().getInfo())

7


In [18]:
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7711, 7801],
   'crs': 'EPSG:32636',
   'crs_transform': [30, 0, 579285, 0, -30, -1961685]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7711, 7801],
   'crs': 'EPSG:32636',
   'crs_transform': [30, 0, 579285, 0, -30, -1961685]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7711, 7801],
   'crs': 'EPSG:32636',
   'crs_transform': [30, 0, 579285, 0, -30, -1961685]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7711, 7801],
   'crs': 'EPSG:32636',
   'crs_transform': [30, 0, 579285, 0, -30, -1961685]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int

In [19]:
landsat.first().get('CLOUD_COVER').getInfo()

84.48

In [20]:
landsat.first().get('DATE_ACQUIRED').getInfo()

'2018-10-05'

In [22]:
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [23]:
landsat_list = landsat.toList(landsat.size())

In [32]:
parameters = {
              'min':5000,
              'max':20000,
              'dimensions': 400, # square size in pixels
              'bands' : ['SR_B5','SR_B6','SR_B7']
}

In [33]:
# Empty Container
data = []

# loop through each image and display it
for i in range (landsat.size().getInfo()):

  # to get the date
  date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
  
  # to get the cloud cover
  cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
  
  # Print their properties
  print ('Image #',i,date,'Cloud cover:',cloud)

  # Display the images
  display(Image(url=ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

  # Make it into list
  this_data = [i,date,cloud]

  # append to the empty container
  data.append(this_data)

# then create dataframe
df = pd.DataFrame (data,columns = ['Image #','Date','Cloud Cover'])

Image # 0 2018-10-05 Cloud cover: 84.48


Image # 1 2018-11-06 Cloud cover: 4.48


Image # 2 2018-11-22 Cloud cover: 12.63


Image # 3 2018-12-08 Cloud cover: 6.65


Image # 4 2018-12-24 Cloud cover: 89.22


Image # 5 2019-01-09 Cloud cover: 42.79


Image # 6 2019-01-25 Cloud cover: 69.87


In [29]:
df

,Image #,Date,Cloud Cover
0,0,2018-10-05,84.48
1,1,2018-11-06,4.48
2,2,2018-11-22,12.63
3,3,2018-12-08,6.65
4,4,2018-12-24,89.22
5,5,2019-01-09,42.79
6,6,2019-01-25,69.87


In [30]:
# make a list sequence
landsat_sequence = [1,2,3]

In [31]:
roi = poi.buffer(20000)

In [38]:
parameters = {
              'min':5000,
              'max':20000,
              'dimensions': 400, 
              'bands' : ['SR_B5','SR_B6','SR_B7'],
              'region' :roi
}

In [39]:
for i in landsat_sequence:

  date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

  cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

  print ('Image #',i,date,'Cloud cover:',cloud)

  display (Image(url=ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 1 2018-11-06 Cloud cover: 4.48


Image # 2 2018-11-22 Cloud cover: 12.63


Image # 3 2018-12-08 Cloud cover: 6.65


In [42]:
# NDVI
palette = ['red','yellow','green']

ndvi_params = {
    'min':0,
    'max':0.4,
    'dimensions':400,
    'palette' :palette,
    'regions' : roi
}

In [44]:
for i in landsat_sequence:

  date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

  cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

  print ('Image #',i,date,'Cloud cover:',cloud)

  display (Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5','SR_B4']).getThumbUrl(ndvi_params)))

Image # 1 2018-11-06 Cloud cover: 4.48


Image # 2 2018-11-22 Cloud cover: 12.63


Image # 3 2018-12-08 Cloud cover: 6.65


In [47]:
map = folium.Map(location=[lon,lat])
map

In [48]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [56]:
# Create a map
my_map = folium.Map(location=[lon, lat], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_params, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [54]:
my_map.save('camp.html')